# VizFlow Demo - v0.5.0

Demonstrates **separate trade/alpha presets** and **schema evolution** with real data.

- `trade_preset` for trade files (e.g., `ylin_v20251204`)
- `alpha_preset` for alpha files (e.g., `jyao_v20251114`)

**v0.5.0 Security Fixes:**
- Path traversal validation (date must be 8 digits)
- Explicit error for unsupported file formats
- Zero-price handling in `forward_return()`

In [3]:
import polars as pl
import vizflow as vf
from pathlib import Path

print("VizFlow version:", vf.__version__)

VizFlow version: 0.5.0


## 1. Trade Data (ylin_v20251204 preset)

Load meords trade files with `trade_preset="ylin_v20251204"` - auto-renames 52 columns.

In [4]:
print("PRESETS:", vf.PRESETS.keys())

PRESETS: dict_keys(['ylin_v20251204', 'jyao_v20251114'])


In [5]:
config = vf.Config(
    trade_dir=Path("data/ylin/trade"),
    trade_pattern="{date}.meords",
    trade_preset="ylin_v20251204",
    market="CN",
    trade_schema={
        "order_qty": vf.ColumnSchema(cast_to=pl.Int64),
    },
)
vf.set_config(config)

df_trade = vf.scan_trade("11110101")
print("=== Trade Data (ylin_v20251204 preset) ===")
print(df_trade.select(['ukey', 'order_side', 'order_qty', 'bid_px0', 'ask_px0']).head(5).collect())

=== Trade Data (ylin_v20251204 preset) ===
shape: (5, 5)
┌──────────┬────────────┬───────────┬─────────┬─────────┐
│ ukey     ┆ order_side ┆ order_qty ┆ bid_px0 ┆ ask_px0 │
│ ---      ┆ ---        ┆ ---       ┆ ---     ┆ ---     │
│ i64      ┆ str        ┆ i64       ┆ f64     ┆ f64     │
╞══════════╪════════════╪═══════════╪═════════╪═════════╡
│ 11000408 ┆ Sell       ┆ 1500      ┆ 22.35   ┆ 22.36   │
│ 11000408 ┆ Buy        ┆ 12000     ┆ 22.64   ┆ 22.66   │
│ 11000408 ┆ Buy        ┆ 3000      ┆ 22.27   ┆ 22.29   │
│ 11000408 ┆ Sell       ┆ 1000      ┆ 22.36   ┆ 22.37   │
│ 11000408 ┆ Buy        ┆ 1000      ┆ 22.45   ┆ 22.48   │
└──────────┴────────────┴───────────┴─────────┴─────────┘


## 2. Alpha Data (jyao_v20251114 preset)

Load alpha files with `alpha_preset="jyao_v20251114"` - auto-renames alpha columns.

In [6]:
config = vf.Config(
    alpha_dir=Path("data/jyao/alpha"),
    alpha_pattern="alpha_{date}.feather",
    alpha_preset="jyao_v20251114",
    market="CN",
)
vf.set_config(config)

df_alpha = vf.scan_alpha("11110101")
print("=== Alpha Data (jyao_v20251114 preset) ===")
print(df_alpha.select(['ukey', 'ticktime', 'bid_px0', 'ask_px0', 'x_10s', 'x_60s', 'x_3m', 'x_30m']).head(5).collect())

=== Alpha Data (jyao_v20251114 preset) ===
shape: (5, 8)
┌──────────┬──────────┬─────────┬─────────┬───────────┬───────────┬───────────┬───────────┐
│ ukey     ┆ ticktime ┆ bid_px0 ┆ ask_px0 ┆ x_10s     ┆ x_60s     ┆ x_3m      ┆ x_30m     │
│ ---      ┆ ---      ┆ ---     ┆ ---     ┆ ---       ┆ ---       ┆ ---       ┆ ---       │
│ i64      ┆ i64      ┆ f64     ┆ f64     ┆ f64       ┆ f64       ┆ f64       ┆ f64       │
╞══════════╪══════════╪═════════╪═════════╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 11000408 ┆ 93000000 ┆ 22.5    ┆ 22.52   ┆ -0.000021 ┆ 0.00007   ┆ 0.000505  ┆ -0.000581 │
│ 11000408 ┆ 93003000 ┆ 22.49   ┆ 22.51   ┆ 0.000122  ┆ -0.000381 ┆ -0.000214 ┆ -0.000742 │
│ 11000408 ┆ 93006000 ┆ 22.53   ┆ 22.54   ┆ -0.000034 ┆ 0.000031  ┆ -0.000258 ┆ -0.0006   │
│ 11000408 ┆ 93009000 ┆ 22.54   ┆ 22.56   ┆ -0.00008  ┆ -0.000001 ┆ -0.000115 ┆ 0.000389  │
│ 11000408 ┆ 93012000 ┆ 22.55   ┆ 22.57   ┆ -0.000294 ┆ 0.000055  ┆ 0.000441  ┆ -0.001009 │
└──────────┴──────────┴

## 3. Parse Timestamps

In [7]:
df_alpha = vf.parse_time(df_alpha, timestamp_col="ticktime")
print("=== Parse Timestamps ===")
print(df_alpha.select(['ukey', 'ticktime', 'tod_ticktime', 'elapsed_ticktime']).head(6).collect())

=== Parse Timestamps ===
shape: (6, 4)
┌──────────┬──────────┬──────────────┬──────────────────┐
│ ukey     ┆ ticktime ┆ tod_ticktime ┆ elapsed_ticktime │
│ ---      ┆ ---      ┆ ---          ┆ ---              │
│ i64      ┆ i64      ┆ time         ┆ i64              │
╞══════════╪══════════╪══════════════╪══════════════════╡
│ 11000408 ┆ 93000000 ┆ 09:30:00     ┆ 0                │
│ 11000408 ┆ 93003000 ┆ 09:30:03     ┆ 3000             │
│ 11000408 ┆ 93006000 ┆ 09:30:06     ┆ 6000             │
│ 11000408 ┆ 93009000 ┆ 09:30:09     ┆ 9000             │
│ 11000408 ┆ 93012000 ┆ 09:30:12     ┆ 12000            │
│ 11000408 ┆ 93015000 ┆ 09:30:15     ┆ 15000            │
└──────────┴──────────┴──────────────┴──────────────────┘


## 4. Aggregation

In [8]:
metrics = {
    "count": pl.len(),
    "avg_x_10s": pl.col("x_10s").mean(),
    "avg_x_3m": pl.col("x_3m").mean(),
}
agg_df = vf.aggregate(df_alpha, group_by=["ukey"], metrics=metrics)
print("=== Aggregation ===")
print(agg_df.collect())

=== Aggregation ===
shape: (3, 4)
┌──────────┬───────┬───────────┬───────────┐
│ ukey     ┆ count ┆ avg_x_10s ┆ avg_x_3m  │
│ ---      ┆ ---   ┆ ---       ┆ ---       │
│ i64      ┆ u32   ┆ f64       ┆ f64       │
╞══════════╪═══════╪═══════════╪═══════════╡
│ 11000739 ┆ 4800  ┆ 0.000001  ┆ -0.000004 │
│ 11002405 ┆ 4800  ┆ -0.000001 ┆ 0.000002  │
│ 11000408 ┆ 4800  ┆ -0.000001 ┆ 0.000006  │
└──────────┴───────┴───────────┴───────────┘
